In [102]:
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import math as m
import soundfile as sf

path_to_folder = os.getcwd()
path_to_voice = '\\LibriSpeech\\train-clean-100'
path_to_plane = '\\recordings\\'

In [103]:
def random_file():
    path = path_to_folder+'\\LibriSpeech\\train-clean-100'
    dirr = os.listdir(path)
    dir1 = random.sample(dirr,1)[0]
    path = path + "\\" + dir1
    dir2 = os.listdir(path)
    dir2 = random.sample(dir2,1)[0]
    path = path + "\\" + dir2
    file = os.listdir(path)
    file = random.sample(file,1)[0]
    while file[-1] != 'c':
        file = os.listdir(path)
        file = random.sample(dir2,1)[0]
    path = path + "\\" + file
    return(path,file)

In [104]:
def random_plane():
    n = random.randint(0,24)
    return(path_to_folder+path_to_plane+"record_"+str(n)+".wav")

In [105]:
path,file = random_file()
print(path,file[:-5])
plane = random_plane()
print(plane)

d:\Utilisateurs\ENAC\projet_AIRBUS\LibriSpeech\train-clean-100\1334\135589\1334-135589-0100.flac 1334-135589-0100
d:\Utilisateurs\ENAC\projet_AIRBUS\recordings\record_15.wav


In [106]:
def get_time(lendata,samplerate):
    return(np.linspace(0,int(lendata/samplerate),lendata))

In [107]:
import soundfile as sf
datavoice, sampleratevoice = sf.read(path)
dataplane, samplerateplane = sf.read(random_plane())
print(len(dataplane),len(datavoice))

5898240 247040


In [108]:
# plt.plot(get_time(datavoice.shape[0],sampleratevoice),datavoice)

In [109]:
def filtre_voix_exp(data,place,sigma,samplerate):
    imax = int(len(data)*place)
    exp = [m.exp(-1*((x-imax)**2)/(2*(sigma**2))) for x in range(len(data))]
    tps = len([0 for k in range(len(data)) if exp[k]>0.001])/samplerate
    res = [exp[k] * data[k] for k in range(len(data))]
    return(res,tps)

In [110]:
# voix = filtre_voix_exp(datavoice,0.5,20000)
# plt.plot(voix)

In [111]:
def filtre_voix_lin(data,sr,place,tps):
    imax_int = int(len(data)*place)
    ideb = imax_int - int(tps*sr/2)
    ifin = imax_int + int(tps*sr/2)
    lin = []
    for i in range(len(data)):
        if i<ideb:
            lin.append(0)
        if i>ideb and i<imax_int:
            lin.append((i-ideb)/(imax_int-ideb))
        if i>imax_int and i<ifin:
            lin.append((ifin-i)/(ifin-imax_int))
        else:
            lin.append(0)
    res = [lin[k] * datavoice[k] for k in range(len(data))]
    return(res)

In [112]:
# voix = filtre_voix_lin(datavoice,sampleratevoice,0.9,4)
# plt.plot(voix)

In [113]:
# plt.plot(get_time(dataplane.shape[0],samplerateplane),dataplane)

In [114]:
#Les enregistrements d'avions et de voix n'ont pas le même sample rate. Afin de pouvoir les additionner facilement, on peut
#modifier le son avec le sample rate le plus petit, en considérant l'intensité constante entre les points 
#effectivement enregistrés

def same_sample_rate(d1,sr1,d2,sr2):
    data1 = []
    data2 = []
    if sr1>sr2:
        data1 = [k for k in d1]
        pas = int(sr1/sr2)
        #data2 = [d2[pas*k] for k in range(int(len(d2)/pas))]
        data2 = []
        for k in range(len(d2)):
            for p in range(pas):
                data2.append(d2[k])
    else:
        data2 = [k for k in d2]
        pas = int(sr2/sr1)
        #data1 = [d1[pas*k] for k in range(int(len(d1)/pas))]
        for k in range(len(d1)):
            for p in range(pas):
                data1.append(d1[k])
    return(np.array(data1),np.array(data2),max([sr1,sr2]))

In [115]:
# da,dv,sr = same_sample_rate(dataplane,samplerateplane,datavoice,sampleratevoice)
# print(len(da),len(dv))


In [116]:
# plt.plot(get_time(dv.shape[0],sr),dv)

In [117]:
# test = []
# i = 0
# fin = len(dv)
# for k in da:
#     if i<fin:
#         test.append(da[i] + dv[i])
#         i+=1
#     else:
#         test.append(da[i])
#         i=i+1


In [118]:
def add_sounds(d1,d2,place):
    res = []
    ld1 = len(d1)
    ld2 = len(d2)
    lmax = max([ld1,ld2])
    lmin = min([ld1,ld2])
    ideb = int(place*lmax)
    ifin = ideb + lmin 
    if ifin>=lmax:
        ifin = lmax-1
    if ld1==lmax:
        i1 = 0
        i2 = 0
        for k in d1:
            if i1>ideb and i1<ifin:
                res.append(d1[i1] + d2[i2])
                i1 += 1
                i2 += 1
            else:
                res.append(d1[i1])
                i1 += 1
    else:
        i1 = 0
        i2 = 0
        for k in d2:
            if i2>ideb and i2<ifin:
                res.append(d1[i1] + d2[i2])
                i1 += 1
                i2 += 1
            else:
                res.append(d2[i2])
                i2 += 1
    return(res)

In [119]:
# add = add_sounds(da,dv,0.15)

In [120]:
# plt.plot(get_time(da.shape[0],sr),add)

In [121]:
# sf.write('test.flac', add, sr)

In [122]:
def generate_database(nb):
    for i in range(nb):
        path,file = random_file()
        path_plane = random_plane()
        datavoice, sampleratevoice = sf.read(path)
        dataplane, samplerateplane = sf.read(path_plane)
        timevoice = len(datavoice)/sampleratevoice
        rand_filtre = random.randint(0,2)
        if rand_filtre==1:
            pos = random.randint(0,100)/100
            datavoice,temps = filtre_voix_exp(datavoice,pos,random.randint(10000,20000),sampleratevoice)
        if rand_filtre == 2:
            pos = random.randint(0,100)/100
            temps = random.randint(2,int(timevoice/2)) 
            try:
                datavoice = filtre_voix_lin(datavoice,sampleratevoice,pos,temps)
                # print("coucou")
            except:
                print(pos,temps,timevoice)
                continue
        plane,voice,sr = same_sample_rate(dataplane,samplerateplane,datavoice,sampleratevoice)
        place = random.randint(0,100)/100
        add = add_sounds(plane,voice,place)
        place = place*len(dataplane)/samplerateplane #date de début de la voix et temps de la voix sur l'enregistrement
        name = "db\\" + str(place) + "--" + str(temps) + ".wav"
        sf.write(name, add, sr)

In [123]:
generate_database(25)

KeyboardInterrupt: 